In [74]:
import requests
import pandas as pd
import gzip
import shutil

# 파일 URL 및 로컬 저장 경로 설정
url = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master/data/amazon-product-reviews/reviews_5_balanced.json.gz'
local_filename = 'reviews_5_balanced.json.gz'

# 파일 다운로드
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(local_filename, 'wb') as f:
        f.write(response.raw.read())
    print(f"파일이 성공적으로 다운로드되었습니다: {local_filename}")
else:
    print(f"파일 다운로드 실패: 상태 코드 {response.status_code}")

# 압축 해제된 파일 경로
json_filename = 'reviews_5_balanced.json'

# gzip 파일 압축 해제
with gzip.open(local_filename, 'rb') as f_in:
    with open(json_filename, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print(f"파일이 성공적으로 압축 해제되었습니다: {json_filename}")

# JSON 파일을 데이터프레임으로 로드
try:
    df = pd.read_json(json_filename, lines=True)
    print("데이터프레임이 성공적으로 생성되었습니다.")
    # 데이터프레임의 첫 몇 행을 출력하여 확인
    print(df.head())
except Exception as e:
    print(f"데이터프레임 생성 중 오류 발생: {e}")

파일이 성공적으로 다운로드되었습니다: reviews_5_balanced.json.gz
파일이 성공적으로 압축 해제되었습니다: reviews_5_balanced.json
데이터프레임이 성공적으로 생성되었습니다.
   overall  verified   reviewTime      reviewerID        asin  \
0        1      True  03 12, 2018  A3QY3THQ42WSCQ  B000YFSR5G   
1        1      True  03 12, 2018  A3QY3THQ42WSCQ  B000YFSR4W   
2        1      True   02 8, 2017  A21HH0VIBKK80J  B000YFSR5G   
3        1      True   02 8, 2017  A21HH0VIBKK80J  B000YFSR4W   
4        1      True  02 19, 2018  A276HQXYS553QW  B0014F8TIU   

                                          reviewText  \
0                                     Waaaay too BIG   
1                                     Waaaay too BIG   
2  Was terribly disappointed, the pants were way ...   
3  Was terribly disappointed, the pants were way ...   
4                              Constantly rolls down   

                                             summary  unixReviewTime  
0                                           One Star      1520812800  
1            

In [75]:
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
import nltk
nltk.download('opinion_lexicon')
print("Total number of words in opinion lexicon", len(opinion_lexicon.words()))
print("Examples of positive words in opinion lexicon", opinion_lexicon.positive()[:8])
print("Examples of negative words in opinion lexicon", opinion_lexicon.negative()[:8])

Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination']


[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [76]:
import nltk
# 리뷰 텍스트를 채점하는 데 사용할 파이썬 사전을 만듬.
df.rename(columns={"reviewText":"text"}, inplace=True)
pos_score = 1
neg_score = -1
word_dict = {}

# 사전에 긍정적 단어 추가
for word in opinion_lexicon.positive():
  word_dict[word] = pos_score

# 사전에 부정적 단어 추가
for word in opinion_lexicon.negative():
  word_dict[word] = neg_score

def bing_liu_score(text):
  sentiment_score = 0
  bag_of_words = word_tokenize(text.lower())
  for word in bag_of_words:
    if word in word_dict:
      sentiment_score += word_dict[word]
  return sentiment_score / len(bag_of_words)

In [77]:
df['bing_liu_score'] = df['text'].apply(bing_liu_score)

In [78]:
df[['asin','text','bing_liu_score']].sample(2)

,asin,text,bing_liu_score
207828,B0015PMQDI,works,1.0
107882,B0002GLCRC,i found this item to exspensive for just one i...,0.0


In [79]:
from sklearn import preprocessing
df['bing_liu_score'] = preprocessing.scale(df['bing_liu_score'])
df.groupby('overall').agg({'bing_liu_score':'mean'})

,bing_liu_score
overall,
1,-0.587784
2,-0.427184
4,0.345290
5,0.529737


## 어휘 기반 접근 방식의 단점
  - 1. 어휘집의 크기에 제한된다. 선택한 어휘집에 단어가 하나도 없으면 이 리뷰에 대한 감성점수를 결정하는 동안 이 정보를 사용할 수 없다.

  - 2. 선택된 어휘집이 표준이라는 가정하에 저자가 제공한 감성 점수/극성을 신뢰


## 지도학습의 접근법
  - 지도학습을 이용한 접근 방식은 데이터의 패턴을 모델링하고 현실에 가까운 예측 함수를 생성할 수 있어 유용.




## 데이터 셋팅

In [80]:
# 제품 등급을 기반으로 0 또는 1의 레이블 정보를 가진 새로운 클래스 레이블 할당하기
df['sentiment'] = 0
df.loc[df['overall']> 3, 'sentiment'] = 1
df.loc[df['overall']< 3, 'sentiment'] = 0

# 간단한 데이터프레임을 유지하기 위해 불필요한 열 제거하기
df.drop(columns=['reviewTime','unixReviewTime','overall','reviewerID','summary'], inplace=True)
df.sample(3)

,verified,asin,text,bing_liu_score,sentiment
3807,True,B00HVV8MOM,"terrible., not like the pic",0.112905,0
32517,True,B015N3NR5I,to fat,-2.045415,0
8525,True,B000B8I61G,muy malo,-0.426675,0


In [81]:
import re

def clean(text):
    """
    텍스트 전처리 함수
    - 소문자 변환
    - 특수 문자 제거
    - 여러 공백 제거
    """
    text = text.lower()  # 소문자로 변환
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # 특수 문자 제거
    text = re.sub(r'\s+', ' ', text)  # 여러 공백 제거
    return text.strip()

In [82]:
## 텍스트 데이터 벡터화 및 지도 학습 알고리즘 적용
df['text_orig'] = df['text'].copy()
df['text'] = df['text'].apply(clean)

## 훈련-테스트 분할

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],
                                                    df['sentiment'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df['sentiment'])

print("size of Training Data:", X_train.shape[0])
print("Size of Test Data:", X_test.shape[0])

print("Distribution of classes in Training Data:")
print("Positive Sentiment", str(sum(Y_train == 1)/ len(Y_train)) * 100)
print("Negative Sentiment", str(sum(Y_train == 0)/ len(Y_train)) * 100)
print("------------------------------------------------------------------------------")
print("Distribution of classes in Testing Data:")
print("Positive Sentiment", str(sum(Y_test == 1)/ len(Y_test)) * 100)
print("Negative Sentiment", str(sum(Y_test == 0)/ len(Y_test)) * 100)

size of Training Data: 235392
Size of Test Data: 58848
Distribution of classes in Training Data:
Positive Sentiment 0.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.50978792822185970.

## 텍스트 벡터화

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df=10, ngram_range=(1,1))
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

## 머신러닝 모델 훈련

In [85]:
from sklearn.svm import LinearSVC

model1 = LinearSVC(random_state=42, tol=1e-5)
model1.fit(X_train_tf, Y_train)

LinearSVC(random_state=42, tol=1e-05)

In [86]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

Y_pred = model1.predict(X_test_tf)
print("Accuracy Score:",accuracy_score(Y_test, Y_pred))
print("ROC-AUC-Score:", roc_auc_score(Y_test, Y_pred))

Accuracy Score: 0.9130641653072322
ROC-AUC-Score: 0.913061453133666


In [87]:
sample_reviews = df.sample(5)
sample_reviews_tf = tfidf.transform(sample_reviews['text'])
sentiment_predictions = model1.predict(sample_reviews_tf)
sentiment_predictions = pd.DataFrame(data=sentiment_predictions, index=sample_reviews.index,
                                     columns=['sentiment_prediction'])
sample_reviews = pd.concat([sample_reviews, sentiment_predictions], axis=1)
print("some sample reviews with their sentiment - ")
sample_reviews[["text_orig","sentiment_prediction"]]

some sample reviews with their sentiment - 


,text_orig,sentiment_prediction
259272,Pro:It fit my Tundra just right\n\nCon:has a l...,1
101645,Dry and went bad faster than most breads in my...,0
113748,"The note pads are really thin, and tear off re...",0
148070,Great deal for these,1
252370,Love ghee...this stuff is notch up,1


In [88]:
sample_reviews_tf

<5x6659 sparse matrix of type '<class 'numpy.float64'>'
	with 44 stored elements in Compressed Sparse Row format>

## 딥러닝을 활용한 사전 훈련된 모델
  - 언어 모델은 문장의 구조와 그 안의 단어를 이해할 수 있도록 자연어를 수학적으로 나타낸 것이다. 언어 모델에는 여러 유형이 있지만 이번에는 사전 훈련된 언어 모델의 사용에 중점을 둔다. 언어 모델의 가장 중요한 특성은 심층 신경망 아키텍처를 사용하고 방대한 데이터 말뭉치에서 훈련된다는 것이다.

## 딥러닝 및 전이 학습
  - 전이학습: 사전 훈련되고 널리 사용 가능한 언어 모델의 이점을 취하기 위해 특정 사용 사례에 모델을 전이해 해당 문제를 풀려는 딥러닝 기술의 하나이다. 또한 한 작업에서 얻은 지식과 정보를 다른 문제에서도 적용할 수 있는 능력을 준다.
  심층신경망은 대규모 텍스트 말뭉치로 훈련. 선택된 모델 아키텍처는 LSTM 또는 transformer가 변형.
  이러한 모델의 훈련에서는 문장에서 한 단어가 제거된 후 문장에 있는 그 외 단어가 주어지면 마스크된 단어를 예측

# Bert
  ## - bert는 트랜스포머 아키텍처를 사용하고 대량의 텍스트 데이터를 사용해 훈련. 여기서 사용한 모델은 Masked Language Model을 사용해 영어 위키백과 말뭉치로 훈련되었다. 마스킹된 언어 모델은 입력에서 일부 토큰을 무작위로 마스킹하는데, 목적은 컨텍스트만을 기반으로 마스킹된 단어의 원래 어휘 ID를 예측하는 것이다. 이 모델은 양 방향이기 때문에 각 문장을 양방향으로 보고 문맥을 훨씬 더 잘 이해할 수 있다. 그리고 버트는 하위 단어를 토큰으로 사용하므로 단어의 의미를 식별할 때 더 세분화 될 수 있다.

# 1. 모델 로드 및 토큰화

In [89]:
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

config = BertConfig.from_pretrained('bert-base-uncased', finetuning_task='binary')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [90]:
def get_tokens(text, tokenizer, max_seq_length, add_special_tokens=True):
  input_ids = tokenizer.encode(text,
                               add_special_tokens=add_special_tokens,
                               max_length = max_seq_length,
                               pad_to_max_length=True)
  attention_mask = [int(id > 0 ) for id in input_ids]
  assert len(input_ids) == max_seq_length
  assert len(attention_mask) == max_seq_length
  return (input_ids, attention_mask)

In [91]:
text = "Here is the sentence I want embeddings for."
input_ids, attention_mask = get_tokens(text,
                                       tokenizer,
                                       max_seq_length=30,
                                       add_special_tokens=True)
input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(text)
print("input_tokens:",input_tokens)
print("input_ids:",input_ids)
print("attention_mask:",attention_mask)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Here is the sentence I want embeddings for.
input_tokens: ['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
input_ids: [101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [92]:
df = df.sample(frac=1).reset_index(drop=True)

In [93]:
df = df[:80000]

In [94]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text_orig'],
                                                    df['sentiment'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df['sentiment'])

X_train_tokens = X_train.apply(get_tokens, args=(tokenizer, 50))
X_test_tokens = X_test.apply(get_tokens, args=(tokenizer, 50))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [95]:
import torch
from torch.utils.data import TensorDataset

input_ids_train = torch.tensor([features[0] for features in X_train_tokens.values], dtype=torch.long)
input_masks_train = torch.tensor([features[1] for features in X_train_tokens.values], dtype=torch.long)
label_ids_train = torch.tensor(Y_train.values, dtype=torch.long)

In [96]:
print(input_ids_train.shape)
print(input_masks_train.shape)
print(label_ids_train.shape)

torch.Size([64000, 50])
torch.Size([64000, 50])
torch.Size([64000])


In [97]:
input_ids_train[2]

tensor([  101, 13695,  2125,  1996,  3332,  2012,  3307, 10898,  2006,  2249,
         9353,  4648,  9108,  2595,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [98]:
train_dataset = TensorDataset(input_ids_train,input_masks_train,label_ids_train)

In [99]:
import torch
from torch.utils.data import TensorDataset

input_ids_test = torch.tensor([features[0] for features in X_test_tokens.values], dtype=torch.long)
input_masks_test = torch.tensor([features[1] for features in X_test_tokens.values], dtype=torch.long)
label_ids_test = torch.tensor(Y_test.values, dtype=torch.long)

In [100]:
print(input_ids_test.shape)
print(input_masks_test.shape)
print(label_ids_test.shape)

torch.Size([16000, 50])
torch.Size([16000, 50])
torch.Size([16000])


In [101]:
test_dataset = TensorDataset(input_ids_test,input_masks_test,label_ids_test)

In [102]:
from torch.utils.data import DataLoader, RandomSampler

train_batch_size = 64
num_train_epochs = 4

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
                              sampler=train_sampler,
                              batch_size=train_batch_size)
t_total = len(train_dataloader) * num_train_epochs

print ("Num training examples = ", len(train_dataset))
print ("Train batch size  = ", train_batch_size)
print ("Num training steps in an epoch = ", len(train_dataloader))
print ("Num Epochs = ", num_train_epochs)
print ("Total num training steps = ", t_total)

Num training examples =  64000
Train batch size  =  64
Num training steps in an epoch =  1000
Num Epochs =  4
Total num training steps =  4000


## 모델 훈련

In [108]:
from torch.utils.data import DataLoader, RandomSampler

train_batch_size = 64
num_train_epochs = 4

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
                              sampler=train_sampler,
                              batch_size=train_batch_size)
t_total = len(train_dataloader) // num_train_epochs

In [109]:
print("Num examples = ", len(train_dataset))
print("Num Epochs = ", num_train_epochs)
print("Total train batch size = ", train_batch_size)
print("Total optimization steps = ", t_total)

Num examples =  64000
Num Epochs =  4
Total train batch size =  64
Total optimization steps =  250


In [110]:
from transformers import AdamW, get_linear_schedule_with_warmup

learning_rate = 1e-4
adam_epsilon = 1e-8
warmup_steps = 0

optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = t_total)

In [111]:
scheduler

In [112]:
from tqdm import trange, notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator = trange(num_train_epochs, desc="Epoch")

# 모델을 '훈련' 모드로 전환한다.
for epoch in train_iterator:
  epoch_iterator = notebook.tqdm(train_dataloader, desc="Iteration")
  for step, batch in enumerate(epoch_iterator):

    # 매 반복마다 모든 그레이디언트 값을 재설정한다.
    model.zero_grad()

    # 모델과 입력 값을 GPU에 넣는다.
    model.to(device)
    batch = tuple(t.to(device) for t in batch)

    # 모델에 넣을 입력 값을 설정한다.
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]}

    # 포워드 패스로 모델을 통과한다. 입력 -> 모델 -> |출력|
    outputs = model(**inputs)

    # 계산된 편차(손실)을 얻는다.
    loss = outputs[0]
    print("\r%f" % loss, end='')

    # 손실을 역전파(기울기 자동 계산)한다.
    loss.backward()

    # 그레이디언트의 최대값을 1.0으로 제한해 그레이디언트 폭발을 방지한다.
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # 매개변수 및 학습률을 업데이트한다.
    optimizer.step()
    scheduler.step()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1000 [00:00<?, ?it/s]

0.155719

Epoch:  25%|██▌       | 1/4 [07:59<23:58, 479.44s/it]

Iteration:   0%|          | 0/1000 [00:00<?, ?it/s]

...

0.175520

Epoch:  75%|███████▌  | 3/4 [23:56<07:58, 478.75s/it]

Iteration:   0%|          | 0/1000 [00:00<?, ?it/s]

0.125780

Epoch: 100%|██████████| 4/4 [31:55<00:00, 478.78s/it]


In [113]:
model.save_pretrained('outputs')

# 모델 평가

In [115]:
import numpy as np
from torch.utils.data import SequentialSampler

test_batch_size = 64
test_sampler =SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset,
                             sampler=test_sampler,
                             batch_size=test_batch_size)
# 이전에 저장한 사전 훈련된 모델을 로드한다.
model = model.from_pretrained('./outputs')

# 예측 및 실제 레이블을 초기화한다.
preds = None
out_label_ids = None

# 모델을 평가 모드로 설정한다.
model.eval()

for batch in notebook.tqdm(test_dataloader, desc='Evaluating'):
  # 모델과 입력 데이터를 gpu에 넣는다.
  model.to(device)
  batch = tuple(t.to(device) for t in batch)

  # '평가'모드 이후로 어떠한 그레이디언트도 추적x
  with torch.no_grad():
    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels':batch[2]}

    # 모델에 입력 데이터를 전달한다.
    outputs = model(**inputs)

    # 레이블을 제공했기 때문에 손실이 발생한다.
    tmp_eval_loss, logits = outputs[:2]

    # 테스트 데이터셋에 항목 배치가 두 개 이상 있을 수 있다.
    if preds is None:
      preds = logits.detach().cpu().numpy()
      out_label_ids = inputs['labels'].detach().cpu().numpy()
    else:
      preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
      out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(),axis=0)

# 최종 손실, 예측 및 정확도를 얻는다.
preds = np.argmax(preds, axis=1)
acc_score = accuracy_score(preds, out_label_ids)
print("Accuracy Score:", acc_score)

Evaluating:   0%|          | 0/250 [00:00<?, ?it/s]

Accuracy Score: 0.9405625


# 결론: 정확도 94% 달성